In [416]:
import spacy
import pandas as pd
from spacy.matcher import PhraseMatcher
from spacy.tokens import Doc

In [376]:
df = pd.read_csv("C:/Users/dschw/Documents/GitHub/Thesis/Output/NCCR_combined_corpus_DE_wording_available_TEST.csv")

In [394]:
nlp = spacy.load("de_core_news_lg", exclude=['tok2vec', 'tagger', 'morphologizer', 'parser',
                         'attribute_ruler', 'lemmatizer'])

nlp.add_pipe("sentencizer")



In [395]:
def standardize_text(text):
    # Replace special characters #todo: simplify replacement
    text = text.replace("ä", "ae").replace("ü", "ue").replace("ö", "oe").replace("Ö", "Oe").replace("Ä", "Ae").replace("Ü", "Ue").replace("ß", "ss").replace("@", "").replace("/", "")
    text = text.replace("ç", "c"). replace("", "")
    
    text = text.lower()
    text = " ".join(text.split())

    return text

In [396]:
def standardize_wording(wording):
    # Replace special characters #todo: simplify replacement
    wording = wording.replace("/", "").replace("@", "").replace("<ord:65430>", "")
    wording = " ".join(wording.split())

    return wording

In [397]:
df['text_prep'] = df['text_prep'].apply(lambda x: standardize_text(x))

In [398]:
df['Wording'] = df['Wording'].apply(lambda x: standardize_wording(x))

In [400]:
df['text_prep'] 

0      "bucher fordert ehrenerklaerung von allen parl...
1      na berlin (ots) - der rheinland-pfaelzische fi...
2      wallner: gruene transparenz-offensive soll kla...
3      zum geheimdienstskandal weltweit umfassender a...
4      "genug gezahlt - ueberstunden steuerfrei", "jo...
                             ...                        
944    na berlin (ots) - buendnis 90die gruenen haben...
945    na berlin. vor der parlamentswahl in simbabwe ...
946    na muenchen die 600 plaetze der festhalle des ...
947    bilaterale ja eu-beitritt nein. dank den bilat...
948    na bemaengelte oekonomisierung von ehe und fam...
Name: text_prep, Length: 949, dtype: object

In [401]:
df['Wording'] 

0      bucher fordert ehrenerklaerung von allen parla...
1      sein parteikollege, der rheinland-pfaelzische ...
2      das team stronach hat bis april 2013 laut rech...
3                                                    wir
4      waehrend andere parteien mit nackter haut vers...
                             ...                        
944                                              gruenen
945    die fdp-bundestagsfraktion fordert alle staatl...
946                                      die altparteien
947                                                  svp
948                                        norbert bluem
Name: Wording, Length: 949, dtype: object

In [425]:
df['doc'] = list(nlp.pipe(df['text_prep']))
df['wording_doc'] = list(nlp.pipe(df['Wording']))




In [424]:
doc = nlp('Das ist ein Test der das Ganze prüft')
get_sub_wording(doc)

Das ist ein Test der 

In [436]:
def get_sub_wording(wording):
    doc2 = Doc(wording.vocab, words=[t.text for i, t in enumerate(wording) if i <3])
    return doc2 
    


In [437]:
df['wording_doc'] = df['wording_doc'].apply(lambda x: get_sub_wording(x))

In [438]:
df['wording_doc']

0          (bucher, fordert, ehrenerklaerung)
1                    (sein, parteikollege, ,)
2                       (das, team, stronach)
3                                       (wir)
4                (waehrend, andere, parteien)
                        ...                  
944                                 (gruenen)
945    (die, fdp-bundestagsfraktion, fordert)
946                        (die, altparteien)
947                                     (svp)
948                          (norbert, bluem)
Name: wording_doc, Length: 949, dtype: object

In [439]:
# Define function to find index of tokens that match Wording content
def get_matches(doc, wording):
    
    # Define Spacy Matcher
    matcher = PhraseMatcher(nlp.vocab)
    # Add patterns from nlp-preprocessed Wording column
    matcher.add("WORDING", [wording])
    
    # Get matches
    matches = matcher(doc)

    return matches



In [440]:
df['matches_new'] = df.apply(lambda x: get_matches(x['doc'], x['wording_doc']), axis=1)

## Fix non-matches

In [441]:
df['matches_new']

0                          [(1212883764967918647, 1, 4)]
1                      [(1212883764967918647, 345, 348)]
2      [(1212883764967918647, 791, 794), (12128837649...
3      [(1212883764967918647, 29, 30), (1212883764967...
4                        [(1212883764967918647, 60, 63)]
                             ...                        
944    [(1212883764967918647, 8, 9), (121288376496791...
945                      [(1212883764967918647, 68, 71)]
946                    [(1212883764967918647, 117, 119)]
947                      [(1212883764967918647, 97, 98)]
948                      [(1212883764967918647, 19, 21)]
Name: matches_new, Length: 949, dtype: object

In [442]:
non = df[~df['matches_new'].astype(bool)]
non = non[['ID', 'doc', 'Wording', 'matches_new', 'wording_segments']]
#non['wording_doc'] = list(nlp.pipe(non['Wording']))
non['doc_tokens'] = non['doc'].apply(lambda x: [token.text for token in x])
non['wording_tokens'] = non['wording_doc'].apply(lambda x: [token.text for token in x])

In [443]:
len(non)

47

In [444]:
non['Wording'].iloc[1]

'volksparte'

In [446]:
non['doc'].iloc[1]

1.6. oesterreich und europa. europaeische union. die volkspartei sieht in der zukuenftigen mitgliedschaft oesterreichs zur europaeischen union die wesentlichste voraussetzung zum ausbau unserer, wirtschaftskraft und zur sicherung der beschaeftigung in oesterreich. der zielstrebigkeit der volkspartei ist es zu verdanken, dass oesterreich am 17. juli 1989 den antrag auf beitritt zu den europaeischen gemeinschaften gestellt hat. am 12. juni 1994 haben die oesterreicherinnen und oesterreicher mit ueberwaeltigender mehrheit entschieden, dass oesterreich mit beginn des kommenden jahres der europaeischen union als gleichberechtigter partner angehoeren wird. damit wurde fuer oesterreich die wichtigste entscheidung seit dem abschluss des staatsvertrag zum wohle unserer heimat getroffen. die antwort der europaeischen union auf die weltweiten herausforderungen, vor allem im technischen und wirtschaftlichen bereich, sind der binnenmarkt und die einheitliche europaeische akte sowie deren weiterentw

In [447]:
print(str(non.iloc[2]['doc_tokens']))

['liberal-sozial', ':-)']


In [448]:
print(str(non.iloc[2]['wording_tokens']))

['barbara', 'schmid-federer']


In [ ]:
--> try match first 5 tokens